# step1.获取数据: 简单存为csv_table.csv文件

In [3]:
from langconv import *
import pandas as pd
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',1000)


In [4]:
def get_company_name_list():
    lines=open('company_cn.txt','r',encoding='utf-8').readlines()
    list_data=[]
    print(len(lines))

    for i in lines:
        i = i.strip('\n')
        list_data.append(i)

    return list_data

list_data=get_company_name_list()

def save_to_csv(list_data):

    # %u7B80%u79F0 简称
    # [\u4e00-\u9fa5] 中文所有字
    name_list=[]
    result_list=[]
    wikipedia.set_lang('zh')

    for company_name in list_data:
        try:
            sentence = wikipedia.summary(company_name, sentences=1, auto_suggest=False)
            name_list.append(company_name)
            result_list.append(sentence)
            
        except Exception as e:
            continue
    result={'name':name_list,'result':result_list}
    df = pd.DataFrame(result)
    df.to_csv('csv_table.csv')
    return 0

3590


In [5]:
import os
import pandas as pd

if not os.path.exists('csv_table.csv'):
    save_to_csv(list_data)
else:
    df=pd.read_csv('csv_table.csv')

In [6]:
sentence_list=list(df['result'])


# step2:数据清洗：所有句子繁体转简体

In [7]:
#将所有繁体句子转换为简体句子
new_sentence_list=[]
for item in sentence_list:
    new_item=''
    for inde,ch in enumerate(item):
        if ch>'\u4e00' or '\u9fa5'>ch:
            #print(ch)
            ch=ch.encode(encoding="utf-8")
            ch = Converter('zh-hans').convert(ch.decode('utf-8'))
            ch = ch.encode('utf-8').decode('utf-8')
            #print(ch)
        new_item=new_item+ch
    #print(new_item)
    new_sentence_list.append(new_item)
print(len(new_sentence_list))
name_list=list(df['name'])
print(len(name_list))

409
409


In [8]:
result_dict={'name':name_list,'sentence':new_sentence_list}
clean_df=pd.DataFrame(result_dict)
clean_df.to_csv('clean_data.csv')

# step3:正则匹配简称字段

In [19]:
import warnings
warnings.filterwarnings('ignore')

import wikipedia
import re
import pandas as pd

def get_breif_name(company_name_list,sentence_list):
    # %u7B80%u79F0 简称
    # [\u4e00-\u9fa5] 中文所有字
    result_list=[]
    name_list=[]
    seg_result_list=[]

    for company_name,item in zip(company_name_list,sentence_list):
        try:
            
            item_list_result=[]
            pattern = re.compile('[a-zA-Z]*[简][称].*?[\u4e00-\u9fa5]+[a-zA-Z]*')
            result = re.findall(pattern, item)
            if result==[]:
                continue
            for i in result:
                item_list_result.append(re.sub('[:：]','',i)[2:])
            #print(item_list_result)
            result_list.append(item_list_result)
            name_list.append(company_name)
            
        except Exception as e:
            continue
    
    #分割 和 或 连接词
    for result in result_list:
        item_list=[]
        for item in result:
            item_list=item_list+'和'.join(item.split('或')).split('和')
        
        seg_result_list.append(item_list)

    
    
    #去掉部分引号
    final_result_list=[]
    for item in seg_result_list:
        list_item=[]
        for i in item:
            #print(re.findall('[a-zA-z]*[\u4e00-\u9fa5]+[a-zA-z]*',i))
            list_item=list_item+[i.replace("'",'').replace(',','').replace('“','')]
        final_result_list.append(list_item)

    
    
    result_dict={'name':name_list,'brief':final_result_list}
    #print(result_dict)
    df=pd.DataFrame(result_dict)
    return df

### 查看含有简称字段的条数：

In [10]:

brief_num=0
for index,item in enumerate(new_sentence_list):
    if re.search('[\u7B80][\u79F0]',item):
        brief_num=brief_num+1
    
print(brief_num)

268


### 执行正则提取操作，查看匹配条数是否等于含有简称两个字的item数目

In [20]:
company_name_list = clean_df.iloc[:,0]
sentence_list=clean_df.iloc[:,1]
df=get_breif_name(company_name_list,new_sentence_list)
df

,name,brief
0,平安银行股份有限公司,[平安银行]
1,万科企业股份有限公司,"[万科, 万科集团]"
2,深圳中国农大科技股份有限公司,"[G*ST国农, 深安达]"
3,深圳世纪星源股份有限公司,"[GST星源, 深原野]"
4,中国宝安集团股份有限公司,"[深宝安A, 深宝安]"
5,中国南玻集团股份有限公司,"[南玻A, 南玻B]"
6,康佳集团股份有限公司,"[康佳集团, 康佳]"
7,深圳中恒华发股份有限公司,[深华发A]
8,国药集团一致药业股份有限公司,[一致药业]
9,深圳市桑达实业股份有限公司,[深桑达A]


In [12]:
sentence_list=list(df['brief'])

for item in sentence_list:
    for i in item:
        i.replace("'", "")
        print(i)

平安银行
万科
万科集团
G*ST国农
深安达
GST星源
深原野
深宝安A
深宝安
南玻A
南玻B
康佳集团
康佳
深华发A
一致药业
深桑达A
华联控股
深南电A
*ST大通
深天马A
中兴
ZTE
深圳机场
中信海直
丰原药业
中联重科
申万宏源
金融街
沈阳机床
通程控股
湖北宜化
东阿阿胶
华天酒店
广东省高速公路发展股份
绿景控股
绿景地产
长虹美菱
虹美菱B
广州浪奇
广弘控股
粤美雅等
穗恒运
“中原环保
湖南投资
“湘中意A
江铃
江铃汽车
海螺型材
中国长安汽车集团
上峰水泥
丰乐种业
苏宁环球
美锦能源
国元证券
振华科技
南风化工
国海证券
中航飞机
广发证券
华闻传媒
太钢不锈
国风塑业
北京顺鑫农业股份
海印股份
茂名永业
安凯客车
中鼎股份
现代投资
“湖南高速
钱江摩托
“电广传媒
“电广实业
华菱钢铁
华菱管线
中国重汽
首钢
蓝焰股份
煤气化
众泰汽车
隆平高科
华润三九
河南豫能控股股份
伟星股份
大族激光
永新股份
华信国际
苏宁易购
苏宁云商
达安基因
远光软件
YGSoft
浙江华峰氨纶股份
獐子岛
江苏沙钢股份
安纳达
广电运通
楚江新材
精诚铜业
游族
广百股份
合肥城建
三全食品
三全公司
科大讯飞
大华股份
天威视讯
拓维信息
“友阿股份
神开股份
辉煌科技
博云新材
中电兴发
森源电气
神剑股份
梦洁家纺
省广股份
杭州杭氧股份
深圳市兴森快捷电路科技股份
誉衡药业
中南文化
珠江啤酒
洽洽食品
雷柏科技
海能达
比亚迪
中公教育
捷顺科技
青青稞酒
加加食品
西部证券
长生生物
长春长生
猛狮科技
百洋水产集团股份
国信证券
第一创业证券
第一创业
华西证券
特锐德
北京探路者户外用品股份
甘肃大禹节水集团股份
华谊兄弟传媒集团
深圳新宙邦科技股份
中青宝
东方财富
长信科技
东方日升
华策影视
香雪制药
天舟文化
福建元力活性炭股份
朗源股份
永清环保
千山药机
银禧科技
尔康制药
湖北三丰智能输送装备股份
江苏省交通科学研究院股份
上海安科瑞电气股份
华录百纳
江苏三六五网络股份
江门市科恒实业股份
唐德影视
贝达药业
药石科技
ATL）成立于
浦发银行
白云机场
华能国际
安徽皖通高速公路股份
中国民生银行
民生银行
宝武集团
华能水电
华电
华电国际
中石化
三一重工
招行
北京歌华有线网络股份
黄山旅

In [21]:
df.to_csv('result.csv')

In [14]:
line='康佳集团或康佳和家'


In [15]:
a=[1,2,3]
b=[4,5,6]
c=[7,8]
d=a+b

In [16]:
a=['1','2','3']
','.join(a)

'1,2,3'

In [17]:
268/409

0.6552567237163814

In [18]:
409/3590

0.11392757660167131